Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 0 ns, sys: 5.74 ms, total: 5.74 ms
Wall time: 12.5 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.76/8343/1 Dashboard: http://10.20.0.76:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 358 ms, sys: 15.2 ms, total: 373 ms
Wall time: 2.53 s


[1.4780804447145015,
 1.439185148783924,
 1.1866767282263444,
 1.196027064519567,
 1.7435505426751856,
 1.2969525677941158,
 1.1868889123734192,
 1.79431386637367,
 1.1593181677716173,
 1.692380312322187,
 1.5057224564582041,
 1.3523917989024636,
 1.0156659145035407,
 1.6721877194453274,
 1.3362076487041687,
 1.1140177201028147,
 1.327415615073739,
 1.685138606639474,
 1.4117899559316083,
 1.3354448502865455,
 1.6154942862789945,
 1.1954982851685398,
 1.4823798243100408,
 1.5190071310243363,
 1.1244188059755014,
 1.8891407867750916,
 1.1612208051877448,
 1.0079663182983367,
 1.8617933305270213,
 1.423546242372236,
 1.6227724862107191,
 1.1114506213219344,
 1.8501493245981506,
 1.1327490128047883,
 1.241607847887677,
 1.4930855239652514,
 1.4355492678605675,
 1.366487916493587,
 1.8857216636576961,
 1.0378758960773624,
 1.9672818560378942,
 1.9235672715839152,
 1.1520995595549364,
 1.6310617865271824,
 1.3285950760948806,
 1.416679523349304,
 1.396868338690167,
 1.936755769513185,
 1.14

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.365239379672858,
 1.0337256640980588,
 1.781006274338601,
 1.0649668411305622,
 1.4681900407802462,
 1.3206669625712468,
 1.4441145622702265,
 1.1089473237054066,
 1.2211535550596557,
 1.855818328781283,
 1.7737457170767381,
 1.5380485087011464,
 1.575970108093561,
 1.7639660214273971,
 1.8904985021414262,
 1.8814041971689455,
 1.2039116652940525,
 1.5554315051012269,
 1.7491551988833978,
 1.0185595495470623,
 1.8223724069574296,
 1.237793919047643,
 1.512792487455254,
 1.7298274737953525,
 1.819804164304835,
 1.5870029571805053,
 1.8971477174589686,
 1.2117056788863154,
 1.319614176725557,
 1.4993588138154843,
 1.6786381057784103,
 1.5940549289049033,
 1.8094556780631625,
 1.6718024425370568,
 1.538085838756567,
 1.7886571831711895,
 1.7956948469363527,
 1.7399386709979985,
 1.5395161794644698,
 1.1004448184924702,
 1.145708537553541,
 1.1798951833014775,
 1.522864710627999,
 1.8732648014887099,
 1.7148996716483391,
 1.3459671460858629]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 186 ms, sys: 10.5 ms, total: 197 ms
Wall time: 1.47 s
